TabNet: Attentive Interpretable Tabular Learning (for starter, <a tagret = "_blank" href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/main/notebooks/samples/explanations/ai-explanations-tabnet-algorithm.ipynb">Google Notebook)</a>

All that is pure free Open Source. You can only build and enjoy the ride.<br />
Ever so grateful

example workflow adapts to your data.
takes full benefit of grouped attention for (out-of-the-box one-hot-encoded) categorical features

<hr />

# setup

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os, sys, json

In [ ]:
import pandas as pd

In [ ]:
from dotenv import find_dotenv
find_dotenv()

In [ ]:
# WandB API key
from dotenv import load_dotenv
load_dotenv("../.env")

In [ ]:
os.getcwd()

In [ ]:
# adding custom code to path for easy imports while "ml-pipelines" is not an installed package
new_path = os.path.realpath(os.path.join(os.getcwd(),
                                         "..", "..", "pkg_src"))
if new_path not in sys.path:
    sys.path.append(new_path)
sys.path

In [ ]:
import sys
os.environ['conda_env'] = os.path.basename(sys.prefix)
os.environ['conda_env']

In [ ]:
mf_flow_name = "TabNetHpCvWandbFlow"
os.environ['pipeline_hp_grid'] = "{}"

In [ ]:
import torch
print(torch.cuda.get_device_name(0))
torch.__version__

In [ ]:
from IPython import get_ipython

def reset_jupyter_history():
    """
    reset cell numbering
    """

    import os
    import sqlite3
    from traitlets.config import Config
    from IPython.core.history import HistoryManager
    
    ip = get_ipython()
    
    if ip is not None:
        # Disable history temporarily
        if hasattr(ip, 'history_manager'):
            ip.history_manager.enabled = False
        
        # Get the path to the history database
        hist_file = os.path.expanduser('~/.ipython/profile_default/history.sqlite')
        if hasattr(ip, 'history_manager') and hasattr(ip.history_manager, 'hist_file'):
            hist_file = ip.history_manager.hist_file
        
        if os.path.exists(hist_file):
            # Close any existing connection
            if hasattr(ip, 'history_manager') and hasattr(ip.history_manager, 'db') and ip.history_manager.db is not None:
                ip.history_manager.db.close()
            
            # Truncate the database
            conn = sqlite3.connect(hist_file)
            conn.execute("DELETE FROM history")
            conn.execute("DELETE FROM sqlite_sequence WHERE name='history'")
            conn.commit()
            conn.close()
        
        # Create a new Config object
        c = Config()
        c.HistoryManager.hist_file = hist_file
        
        # Create a new HistoryManager instance
        new_history_manager = HistoryManager(shell=ip, config=c)
        
        # Assign the new history manager to the shell
        ip.history_manager = new_history_manager
        
        # Enable history
        ip.history_manager.enabled = True

        ip.kernel.shell.execution_count = 0
        
        print("Jupyter history has been reset.")
    else:
        print("This function must be run within a Jupyter notebook.")

reset_jupyter_history()
get_ipython().kernel.shell.execution_count

<hr />

# Generate data

In [ ]:
from retrain_pipelines.dataset import DatasetType, pseudo_random_generate

# Define the number of samples
num_samples = 10_000 # 500 # 10 # 1_500 # 

data = pseudo_random_generate(DatasetType.TABULAR_CLASSIFICATION, num_samples)

# Display the first few rows of the dataset
print(data.head())
#data.to_csv(os.path.realpath(os.path.join('..', 'data', 'synthetic_classif_tab_data_4classes.csv')), index=False)

In [ ]:
#from sklearn.preprocessing import StandardScaler, \
#                                  OneHotEncoder
#
#os.environ["ml_pipeline_type"] = "tabnet_classif_torchserve"
#
#from retrain_pipelines.model import preprocess_data_fct
#
#scaler = StandardScaler()
#encoder = OneHotEncoder(sparse_output=False, drop=None)
#pp_data = preprocess_data_fct(data, scaler, encoder,
#                    buckets={'num_feature1': [-1.002, -0.6, -0.19999999999999996, 0.20000000000000018, 0.6000000000000001, 1.0],
#                             'num_feature2': 15},
#                    is_training=True)
#
#del os.environ["ml_pipeline_type"]
#
#display(pp_data)
#display(scaler)
#display(encoder)

In [ ]:
from retrain_pipelines.dataset import features_desc
pd.DataFrame([features_desc(data)])

In [ ]:
from retrain_pipelines.dataset import features_distri_plot
display(features_distri_plot(data))

In [ ]:
from retrain_pipelines.dataset.features_dependencies import dataset_to_heatmap_fig
heatmap_fig, ax = dataset_to_heatmap_fig(data)
heatmap_fig.patch.set_facecolor((1.0, 1.0, 1.0, 0.6))
display(heatmap_fig)

<hr />

# TabNet

In [ ]:
pipeline_hp_grid = {
    "trainer": {
        "max_epochs":[30, 50], #sys.maxsize,
        "patience":[100],
        "batch_size":[1024],
        "virtual_batch_size":[256],
    },
    "model": {
        "n_d":[64],
        "n_a":[64],
        "n_steps":[5],
        "gamma":[1.5],
        "n_independent":[2],
        "n_shared":[2],
        "lambda_sparse":[1e-4],
        "momentum":[0.3],
        "clip_value":[2.],
        "optimizer_fn":["torch.optim.Adam"],
        "optimizer_params":[dict(lr=2e-2)],
        "scheduler_params":[{"gamma": 0.80,
                            "step_size": 20}],
        "scheduler_fn":["torch.optim.lr_scheduler.StepLR"],
        "epsilon":[1e-15]
    }
}

In [ ]:
from textwrap import dedent
#os.environ['pipeline_hp_grid'] = str(json.dumps(dedent("""{pipeline_hp_grid}""".format(pipeline_hp_grid=pipeline_hp_grid))))
#print(os.environ['pipeline_hp_grid'])
os.environ['pipeline_hp_grid'] = str(json.dumps(dedent("""{pipeline_hp_grid}""".format(pipeline_hp_grid=pipeline_hp_grid)))).replace("'", '"').strip('"')
print(os.environ['pipeline_hp_grid'])

In [ ]:
from retrain_pipelines.utils import dict_dict_list_get_all_combinations

# Get all possible combinations
all_hp_combinations = dict_dict_list_get_all_combinations(pipeline_hp_grid)
# pick first
all_hp_combinations[0]

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

if 'target' in data:
    y = data.pop('target')
y.unique()

In [ ]:
from sklearn.preprocessing import StandardScaler, \
                                  OneHotEncoder
os.environ["ml_pipeline_type"] = "mf_tabnet_classif_torchserve"
from ml_pipelines.model import preprocess_data_fct

scaler = StandardScaler()
encoder = OneHotEncoder(sparse_output=False, drop=None)
buckets={'num_feature1': 5}
grouped_features = []
X_transformed = preprocess_data_fct(data, scaler, encoder,
                                    buckets=buckets,
                                    grouped_features=grouped_features,
                                    is_training=True)
len(X_transformed.columns), X_transformed.columns

In [ ]:
clf = TabNetClassifier(
    **all_hp_combinations[0]['model'],
    grouped_features=grouped_features
)

In [ ]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(
    range(len(X_transformed)), test_size=0.2, random_state=0)

X_train = X_transformed.values[train_idx]
y_train = y.values[train_idx]
X_valid = X_transformed.values[test_idx]
y_valid = y.values[test_idx]

In [ ]:
from pytorch_tabnet.callbacks import Callback
import wandb

class PrintLR(Callback):
    def __init__(self):
        self.previous_lr = None

    def on_epoch_end(self, epoch, logs=None):
        current_lr = self.trainer.optimizer_params['lr']
        if self.previous_lr is None or current_lr != self.previous_lr:
            print(f"Epoch {epoch+1}: Learning rate changed to {current_lr:.6f}")
        self.previous_lr = current_lr

class WandbCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log({"epoch": epoch + 1, **logs})

    def on_train_end(self, logs=None):
        wandb.finish()

In [ ]:
#help(clf.fit)

In [ ]:
clf.fit(
    X_train=X_train, y_train=y_train,
    loss_fn=torch.nn.CrossEntropyLoss(),
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    **all_hp_combinations[0]['trainer'],
    augmentations=None,
    drop_last=False,
    callbacks=[PrintLR()]#, WandbCallback]
)

In [ ]:
#clf.history

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.plot(clf.history['loss'])
plt.show()

epochs = range(1, len(clf.history['train_accuracy']) + 1)
train_accuracy = clf.history['train_accuracy']
valid_accuracy = clf.history['valid_accuracy']
learning_rates = clf.history['lr']


fig, ax1 = plt.subplots()

# Plotting train_accuracy and valid_accuracy on the primary y-axis
ax1.plot(epochs, train_accuracy, 'b-', label='Train Accuracy', zorder=2)
ax1.plot(epochs, valid_accuracy, 'g-', label='Validation Accuracy', zorder=2)
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
ax1.tick_params(axis='y', labelcolor='black')

# Create a secondary y-axis for the learning rates
ax2 = ax1.twinx()
ax2.plot(epochs, learning_rates, 'r-', label='Learning Rate', zorder=1)
ax2.set_ylabel('Learning Rate', color='r')
ax2.tick_params(axis='y', labelcolor='r')

# Adding legends (making sure both ar above plots (zorder)
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
first_legend = plt.legend(handles1, labels1, loc='upper right')
ax2.add_artist(first_legend)
ax2.legend(handles2, labels2, loc='upper left')

# Show grid and plot
plt.grid(True)
plt.title('Training and Validation Accuracy with Learning Rates')
plt.show()

In [ ]:
clf.predict_proba(X_valid)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_valid) # I know, measuring model perf' on valid dataset is stooop'
display(y_pred)
test_acc = accuracy_score(y_pred=y_pred, y_true=y_valid)

print(f"BEST VALID SCORE : {clf.best_cost}")
print(f"FINAL TEST SCORE : {test_acc}")

In [ ]:
explain_matrix, masks = clf.explain(X_valid) # change to holdout dataset here too

fig, axs = plt.subplots(1, 5, figsize=(20,20))

for i in range(5):
    axs[i].imshow(masks[i][:50])
    axs[i].set_title(f"mask {i}")

In [ ]:
y_pred

In [ ]:
X_valid.shape

In [ ]:
clf.feature_importances_

In [ ]:
clf.grouped_features

In [ ]:
import inspect
from pytorch_tabnet.callbacks import Callback
inspect.getsource(Callback)

<hr />

# Metaflow Run

Chosse which domain shall be considered for the HP tuning grid search&nbsp;:

In [ ]:
pipeline_hp_grid = {
    "trainer": {
        "max_epochs":[100], #sys.maxsize,
        "patience":[10],
        "batch_size":[1024],
        "virtual_batch_size":[256],
    },
    "model": {
        "n_d":[64],
        "n_a":[64],
        "n_steps":[6],
        "gamma":[1.5],
        "n_independent":[2],
        "n_shared":[2],
        "lambda_sparse":[1e-4],
        "momentum":[0.3],
        "clip_value":[2.],
        "optimizer_fn":["torch.optim.Adam"],
        "optimizer_params":[dict(lr=2e-2), dict(lr=0.1)],
        "scheduler_params":[{"gamma": 0.80,
                            "step_size": 20}],
        "scheduler_fn":["torch.optim.lr_scheduler.StepLR"],
        "epsilon":[1e-15]
    }
}
from textwrap import dedent
os.environ['pipeline_hp_grid'] = str(json.dumps(dedent("""{pipeline_hp_grid}""".format(pipeline_hp_grid=pipeline_hp_grid)))).replace("'", '"').strip('"')
print(os.environ['pipeline_hp_grid'])

## Run flow

In [ ]:
%reload_ext retrain_pipelines.local_launcher_magic

In [ ]:
%retrain_pipelines_local retraining_pipeline.py run --help

In [ ]:
%retrain_pipelines_local retraining_pipeline.py run \
    --data_file "../data/synthetic_classif_tab_data_4classes.csv" \
    --buckets_param '{"num_feature2": 100, "num_feature4": 50}' \
    --pipeline_hp_grid "${pipeline_hp_grid}" \
    --cv_folds 2 \
    --wandb_run_mode disabled
    #--preprocess_artifacts_path "." \
    #--pipeline_card_artifacts_path "." \

In [ ]:
%retrain_pipelines_local retraining_pipeline.py resume pipeline_card

# Inspectors

After the fact inspecting retraining pipelines runs.

## local Metaflow SDK

In [ ]:
from retrain_pipelines.frameworks import local_metaflow as metaflow

In [ ]:
metaflow.Task("TabNetHpCvWandbFlow/973/pipeline_card/29591",  attempt=0)['model'].data

## local custom card explorer

Retrieve the full path to the custom html pipeline card in the local datastore&nbsp;:

In [ ]:
from retrain_pipelines.inspectors import browse_local_pipeline_card

In [ ]:
browse_local_pipeline_card(mf_flow_name)
#browse_local_pipeline_card(mf_run_id=800, verbose=True)

<hr />

# WandB

Make sure to have the `WANDB_API_KEY` environement variable set adequately.<br />
It can be through a `secret`.

<b>programmatically browse the saved source-code</b>

In [ ]:
from retrain_pipelines.inspectors import get_execution_source_code

for source_code_artifact in get_execution_source_code(mf_run_id=860): #mf_flow_name):
    print(f" - {source_code_artifact.name} {source_code_artifact.url}")

<b>The below command will download source-code artifacts for a given run and open a file explorer on the parent dir&nbsp;:</b>

In [ ]:
from retrain_pipelines.inspectors import explore_source_code

explore_source_code(mf_flow_name=mf_flow_name)

<hr />

# dependencies

In [ ]:
%pip list

In [ ]:
%pip list --format=freeze > requirements.txt

# debug zone

In [ ]:
for artifact_name in model_card_task.artifacts:
    print(f"{artifact_name}")

In [ ]:
print(pipeline_card_task['html'].data[:2000])

In [ ]:
custom_card.path

In [ ]:
custom_card.__dict__.keys()

In [ ]:
custom_card._card_ds.get_storage_root(storage_type='local')

In [ ]:
metaflow.metaflow_config.CARD_SUFFIX

In [ ]:
ds_root = pipeline_card_task.metadata_dict.get("ds-root", None)
ds_type, ds_root

In [ ]:
%ls $ds_root/mf.cards/$pipeline_card_task.parent.parent.parent.id/runs/$pipeline_card_task.parent.parent.id/tasks/$pipeline_card_task.parent.task.id/cards/

In [ ]:
pipeline_card_task.metadata_dict

In [ ]:
storage_impl = [d for d in metaflow.plugins.DATASTORES if d.TYPE == ds_type][0]
storage_impl

<hr width=50% />

In [ ]:
for flow in metaflow.Metaflow():
    print(flow)
    for run in flow.runs():
        print(f"  Run ID: {run.id}")
        print(f"  Steps: {[str(step) for step in run.steps()]}")

In [ ]:
flows = list(metaflow.Metaflow())
if flows:
    first_flow = flows[0]

    # Get the last run from the first flow
    last_run = list(first_flow.runs())[0]

    # Print details of the last run
    print(f"Flow: {first_flow.id}")
    print(f"Last Run ID: {last_run.id}")
    print(f"Start Time: {last_run.created_at}")
    print(f"Status: {last_run.successful}")
    for step in last_run.steps():
        print(f"Step: {step.id}")
        print(step.task)